In [1]:
#from fosforio import snowflake
from fosforml import *
from fosforml.constants import MLModelFlavours
from matplotlib import pyplot as plt
import pandas as pd
pd.set_option('display.max_columns', 500)
import seaborn as sns
from sklearn.metrics import mean_absolute_percentage_error
import numpy as np
import warnings; warnings.simplefilter('ignore')
from joblib import dump, load
import requests
from tqdm import tqdm
import time
import calendar

from time import sleep
import configparser
from dateutil.relativedelta import relativedelta
import datetime
from dateutil.easter import easter
from scipy.optimize import minimize_scalar
from scipy.optimize import curve_fit
import holidays, itertools

%matplotlib inline

In [2]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()

In [ ]:
my_session.sql('''
CREATE OR REPLACE PROCEDURE create_pricing_model(table_name STRING)
RETURNS STRING
LANGUAGE PYTHON
RUNTIME_VERSION = '3.8'
PACKAGES = ('snowflake-snowpark-python', 'pandas', 'numpy', 'scipy')
HANDLER = 'create_pricing_model'
AS
$$
from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import col
import pandas as pd
import numpy as np
from scipy.optimize import minimize_scalar, curve_fit, brentq
import itertools

# Main procedure to create pricing model
def create_pricing_model(session, table_name=None):

    # Define the revenue function based on price and demand curve
    def revenue(price):
        return price * demand_curve(price, a_fit, b_fit, c_fit, d_fit, max_demand)

    # Define the demand curve as an exponential function
    def demand_curve(x, a, b, c, d, max_demand):
        demand = a * np.exp(-b * x) + c
        demand = np.where(x <= max_demand, np.minimum(demand, max_demand), demand)
        return demand + d

    # Calculate price based on the number of rooms
    def demand_to_price(num_rooms, a, b, c, d, max_demand):
        def root_func(x):
            return num_rooms - (a * np.exp(-b * x) + c)

        try:
            price = brentq(root_func, 0, 200)
        except ValueError:
            price_range = (0, 200)
            price = np.random.uniform(*price_range)

        return price

    # Retrieve data from Snowflake table and transform it to a Pandas DataFrame
    expanded_df = session.table([table_name]).to_pandas()
    expanded_df.columns = map(lambda x: str(x).lower(), expanded_df.columns)

    # Initialize an empty DataFrame for storing results
    results = pd.DataFrame(columns=['month', 'hotel', 'room_limit', 'room_type', 'dow', 'optimal_rate', 'expected_rn', 'expected_rev', 'optimal_rate_lim_inv'])
    
    hotel_types = ['Resort Hotel', 'City Hotel']
    room_types = ['A', 'D', 'E']

    # Aggregate daily room numbers and merge with ADR frequency data
    daily_rns = expanded_df.groupby(['arrival_date', 'dow', 'month', 'hotel', 'room_type']).agg({'room_limit': 'mean', 'total_rns': 'sum'}).reset_index()
    daily_rns = daily_rns.groupby(['dow', 'month', 'hotel', 'room_type']).agg({'room_limit': 'mean', 'total_rns': ['sum', 'mean', 'median']}).reset_index()
    daily_rns.columns = ['_'.join(col) for col in daily_rns.columns]
    
    adr_frequency = expanded_df.groupby(['dow', 'month', 'adr', 'hotel', 'room_type']).agg({'room_limit': 'mean', 'total_rns': 'sum'})
    adr_frequency.reset_index(inplace=True)
    
    merged_df = pd.merge(adr_frequency, daily_rns, how='left', left_on=['dow', 'month', 'hotel', 'room_type'], right_on=['dow_', 'month_', 'hotel_', 'room_type_'], suffixes=('_act', '_tot'))
    merged_df = merged_df.drop(['dow_', 'month_'], axis=1)

    # Calculate demand probabilities and expected revenue
    merged_df['probability'] = merged_df['total_rns'] / merged_df['total_rns_sum']
    merged_df['expected_rns'] = merged_df['probability'] * merged_df['total_rns_median']
    merged_df = merged_df.sort_values(by=['dow', 'month', 'adr'], ascending=[True, True, False])
    merged_df['expected_demand'] = merged_df.groupby(['dow', 'month'])['expected_rns'].cumsum()
    merged_df['expected_rev'] = merged_df['adr'] * merged_df['expected_demand']

    # Initialize variables for iteration over months and days of the week
    months = merged_df.month.unique()
    dow = merged_df.dow.unique()

    # Loop through hotels, room types, months, and days of the week to calculate optimal prices
    for hotel in hotel_types:
        for room_type in room_types:
            for month in months:
                for day in dow:
                    data_subset = merged_df[(merged_df['dow'] == day) & (merged_df['hotel'] == hotel) & (merged_df['room_type'] == room_type) & (merged_df['month'] == month)].reset_index()
                    if data_subset.empty:
                        continue

                    # Filter out outliers using Z-scores
                    mean = data_subset['adr'].mean()
                    std_dev = data_subset['adr'].std()
                    data_subset['z_scores'] = np.abs((data_subset['adr'] - mean) / std_dev)
                    data_subset = data_subset[data_subset['z_scores'] <= 2]

                    # Prepare data for curve fitting to estimate demand curve parameters
                    x_data = data_subset['adr'].values
                    y_data = data_subset['expected_demand'].values

                    try:
                        initial_guess = [1, 0.01, 1, 1, data_subset['total_rns_median'].values[0]]
                        bounds = ([0, 0, 0, 0, 0], [np.inf, np.inf, np.inf, np.inf, np.inf])
                        maxfev = 10000
                        params, _ = curve_fit(demand_curve, x_data, y_data, bounds=bounds, p0=initial_guess, maxfev=maxfev)
                    except RuntimeError as e:
                        print(f"Error fitting demand curve for {hotel}, {room_type}, {month}, {day}: {e}")
                        continue

                    a_fit, b_fit, c_fit, d_fit, max_demand = params

                    # Calculate optimal price and expected revenue
                    objective = lambda price: -revenue(price)
                    optimize = minimize_scalar(objective, bounds=(45, 200), method='bounded')
                    optimal_price = optimize.x
                    max_revenue = -optimize.fun
                    expected_rns = demand_curve(optimal_price, a_fit, b_fit, c_fit, d_fit, max_demand)
                    optimal_rate_lim_inv = demand_to_price(data_subset['room_limit'].mean(), a_fit, b_fit, c_fit, d_fit, max_demand)

                    # Add the results to the DataFrame
                    new_row = pd.DataFrame({'hotel': hotel, 'room_type': room_type, 'room_limit': data_subset['room_limit'].mean(), 'month': month, 'dow': day, 'optimal_rate': optimal_price, 'expected_rev': max_revenue, 'expected_rn': expected_rns, 'optimal_rate_lim_inv': optimal_rate_lim_inv}, index=[0])
                    results = pd.concat([results, new_row], ignore_index=True)

    # Round off final results
    results['optimal_rate'] = results['optimal_rate'].round()
    results['optimal_rate_lim_inv'] = results['optimal_rate_lim_inv'].round()
    results['expected_rn'] = results['expected_rn'].round().astype(int)
    results['expected_rev'] = results['expected_rev'].round()

    # Generate date ranges and combinations of hotel types and room types for further modeling
    combinations = list(itertools.product(hotel_types, room_types))
    combinations_df = pd.DataFrame(combinations, columns=['hotel', 'room_type'])
    month_dict = {month: index for index, month in enumerate(pd.date_range('2020-01-01', periods=12, freq='M').strftime('%B'), 1)}

    new_data = pd.DataFrame()

    # Create time series data for each combination of hotel and room type across multiple years
    for year in range(2020, 2024):
        for month in month_dict.values():
            start_date = pd.to_datetime(f'{year}-{month}-01').replace(day=1)
            end_date = pd.to_datetime(f'{year}-{month}-01').replace(day=1) + pd.offsets.MonthEnd(0)
            date_range = pd.date_range(start_date, end_date, freq='D')
            df = pd.DataFrame(date_range, columns=['arrival_date'])
            df['dow'] = df['arrival_date'].dt.day_name()
            df['month'] = df['arrival_date'].dt.month_name()

            # Merge the date data with hotel/room type combinations
            result_df = df.assign(key=1).merge(combinations_df.assign(key=1), on='key').drop('key', axis=1)
            new_data = pd.concat([new_data, result_df], ignore_index=True)

    # Finalize the data by merging with the calculated results
    final_data = pd.merge(new_data, results, how='left', on=['dow', 'hotel', 'room_type', 'month'])
    final_data.replace([np.inf, -np.inf], np.nan, inplace=True)
    final_data = final_data.dropna()
    final_data['room_limit'] = final_data['room_limit'].astype(int)
    final_data['arrival_date'] = pd.to_datetime(final_data['arrival_date']).dt.strftime("%Y-%m-%d %H:%M:%S")
    
    # Write the final modeled data back to Snowflake
    df_model = session.createDataFrame(final_data.values.tolist(), schema=final_data.columns.tolist())
    df_model.write.mode("overwrite").save_as_table("TTH_DB.TTH_REV_OPT_Schema.PROCESSED_PRICING_DATA")

    # Expand the booking data to one row per stay date
    expanded_dfc1 = pd.DataFrame()
    
    for _, row in data.iterrows():
        num_stay_dates = row['total_rns']
        try:
            # Create a row for each stay date
            expanded_bookingc1 = pd.DataFrame({
                'hotel': row['hotel'],
                'room_type': row['reserved_room_type'], 
                'market_segment': row['market_segment'],
                'deposit_type':row['deposit_type'],
                'meal': row['meal'],
                'distribution_channel': row['distribution_channel'],
                'customer_type': row['customer_type'],
                'arrival_date': pd.date_range(start=row['arrival_date_transformed'], periods=num_stay_dates),
                'total_rns': 1,
                'adr': row['adr'],
                'room_limit': row['room_limit']
            })
            
            # Append the stay date information to the new dataframe
            expanded_dfc1 = pd.concat([expanded_dfc1, expanded_bookingc1], ignore_index=True)
        except ValueError as e:
            print(f"Error processing booking for {row['hotel']} on {row['arrival_date_transformed']} : {num_stay_dates} {e}")
    
    # Final processing of booking frequency and writing back to Snowflake
    expanded_dfc1 = expanded_dfc1.sort_values('arrival_date')
    expanded_dfc1 = expanded_dfc1.reset_index(drop=True)
    expanded_dfc1['adr']= np.round(expanded_dfc1['adr'], 2)
    expanded_dfc1['dow'] = expanded_dfc1.arrival_date.dt.strftime('%A')
    expanded_dfc1['month'] = expanded_dfc1.arrival_date.dt.strftime('%B')
    
    booking_frequency = expanded_dfc1.groupby(['hotel','room_type','market_segment','deposit_type','meal',
                                            'distribution_channel','customer_type',
                                               'arrival_date'], as_index=False, sort=True).agg({'adr': 'mean','room_limit': 'mean',
                                                                    'total_rns': 'sum'})
    booking_frequency = booking_frequency.sort_values(by=['arrival_date','hotel','room_type'], ascending=[True, True, True])
    
    # Write booking frequency data back to Snowflake
    booking_frequency["arrival_date"] = pd.to_datetime(booking_frequency["arrival_date"]).dt.strftime("%Y-%m-%d %H:%M:%S")
    df_inter=session.createDataFrame(
            booking_frequency.values.tolist(),
            schema=booking_frequency.columns.tolist())
    df_inter.write.mode("overwrite").save_as_table("TTH_DB.TTH_REV_OPT_Schema.booking_frequency")

    return "Data processed and dumped back to Snowflake"
$$
''').collect()


In [8]:
my_session.schema

AttributeError: 'Session' object has no attribute 'schema'

In [7]:
def create_pricing_model(session, table_name=None):

    # Define the revenue function based on price and demand curve
    def revenue(price):
        return price * demand_curve(price, a_fit, b_fit, c_fit, d_fit, max_demand)

    # Define the demand curve as an exponential function
    def demand_curve(x, a, b, c, d, max_demand):
        demand = a * np.exp(-b * x) + c
        demand = np.where(x <= max_demand, np.minimum(demand, max_demand), demand)
        return demand + d

    # Calculate price based on the number of rooms
    def demand_to_price(num_rooms, a, b, c, d, max_demand):
        def root_func(x):
            return num_rooms - (a * np.exp(-b * x) + c)

        try:
            price = brentq(root_func, 0, 200)
        except ValueError:
            price_range = (0, 200)
            price = np.random.uniform(*price_range)

        return price

    # Retrieve data from Snowflake table and transform it to a Pandas DataFrame
    expanded_df = session.table([table_name]).to_pandas()
    expanded_df.columns = map(lambda x: str(x).lower(), expanded_df.columns)

    # Initialize an empty DataFrame for storing results
    results = pd.DataFrame(columns=['month', 'hotel', 'room_limit', 'room_type', 'dow', 'optimal_rate', 'expected_rn', 'expected_rev', 'optimal_rate_lim_inv'])
    
    hotel_types = ['Resort Hotel', 'City Hotel']
    room_types = ['A', 'D', 'E']
    print(expanded_df.columns)
    # Aggregate daily room numbers and merge with ADR frequency data
    daily_rns = expanded_df.groupby(['arrival_date_transformed', 'dow', 'month', 'hotel', 'room_type']).agg({'room_limit': 'mean', 'total_rns': 'sum'}).reset_index()
    daily_rns = daily_rns.groupby(['arrival_date_week_number', 'arrival_date_day_of_month', 'hotel', 'room_type']).agg({'room_limit': 'mean', 'total_rns': ['sum', 'mean', 'median']}).reset_index()
    daily_rns.columns = ['_'.join(col) for col in daily_rns.columns]
    
    adr_frequency = expanded_df.groupby(['dow', 'month', 'adr', 'hotel', 'room_type']).agg({'room_limit': 'mean', 'total_rns': 'sum'})
    adr_frequency.reset_index(inplace=True)
    
    merged_df = pd.merge(adr_frequency, daily_rns, how='left', left_on=['dow', 'month', 'hotel', 'room_type'], right_on=['dow_', 'month_', 'hotel_', 'room_type_'], suffixes=('_act', '_tot'))
    merged_df = merged_df.drop(['dow_', 'month_'], axis=1)

    # Calculate demand probabilities and expected revenue
    merged_df['probability'] = merged_df['total_rns'] / merged_df['total_rns_sum']
    merged_df['expected_rns'] = merged_df['probability'] * merged_df['total_rns_median']
    merged_df = merged_df.sort_values(by=['dow', 'month', 'adr'], ascending=[True, True, False])
    merged_df['expected_demand'] = merged_df.groupby(['dow', 'month'])['expected_rns'].cumsum()
    merged_df['expected_rev'] = merged_df['adr'] * merged_df['expected_demand']

    # Initialize variables for iteration over months and days of the week
    months = merged_df.month.unique()
    dow = merged_df.dow.unique()

    # Loop through hotels, room types, months, and days of the week to calculate optimal prices
    for hotel in hotel_types:
        for room_type in room_types:
            for month in months:
                for day in dow:
                    data_subset = merged_df[(merged_df['dow'] == day) & (merged_df['hotel'] == hotel) & (merged_df['room_type'] == room_type) & (merged_df['month'] == month)].reset_index()
                    if data_subset.empty:
                        continue

                    # Filter out outliers using Z-scores
                    mean = data_subset['adr'].mean()
                    std_dev = data_subset['adr'].std()
                    data_subset['z_scores'] = np.abs((data_subset['adr'] - mean) / std_dev)
                    data_subset = data_subset[data_subset['z_scores'] <= 2]

                    # Prepare data for curve fitting to estimate demand curve parameters
                    x_data = data_subset['adr'].values
                    y_data = data_subset['expected_demand'].values

                    try:
                        initial_guess = [1, 0.01, 1, 1, data_subset['total_rns_median'].values[0]]
                        bounds = ([0, 0, 0, 0, 0], [np.inf, np.inf, np.inf, np.inf, np.inf])
                        maxfev = 10000
                        params, _ = curve_fit(demand_curve, x_data, y_data, bounds=bounds, p0=initial_guess, maxfev=maxfev)
                    except RuntimeError as e:
                        print(f"Error fitting demand curve for {hotel}, {room_type}, {month}, {day}: {e}")
                        continue

                    a_fit, b_fit, c_fit, d_fit, max_demand = params

                    # Calculate optimal price and expected revenue
                    objective = lambda price: -revenue(price)
                    optimize = minimize_scalar(objective, bounds=(45, 200), method='bounded')
                    optimal_price = optimize.x
                    max_revenue = -optimize.fun
                    expected_rns = demand_curve(optimal_price, a_fit, b_fit, c_fit, d_fit, max_demand)
                    optimal_rate_lim_inv = demand_to_price(data_subset['room_limit'].mean(), a_fit, b_fit, c_fit, d_fit, max_demand)

                    # Add the results to the DataFrame
                    new_row = pd.DataFrame({'hotel': hotel, 'room_type': room_type, 'room_limit': data_subset['room_limit'].mean(), 'month': month, 'dow': day, 'optimal_rate': optimal_price, 'expected_rev': max_revenue, 'expected_rn': expected_rns, 'optimal_rate_lim_inv': optimal_rate_lim_inv}, index=[0])
                    results = pd.concat([results, new_row], ignore_index=True)

    # Round off final results
    results['optimal_rate'] = results['optimal_rate'].round()
    results['optimal_rate_lim_inv'] = results['optimal_rate_lim_inv'].round()
    results['expected_rn'] = results['expected_rn'].round().astype(int)
    results['expected_rev'] = results['expected_rev'].round()

    # Generate date ranges and combinations of hotel types and room types for further modeling
    combinations = list(itertools.product(hotel_types, room_types))
    combinations_df = pd.DataFrame(combinations, columns=['hotel', 'room_type'])
    month_dict = {month: index for index, month in enumerate(pd.date_range('2020-01-01', periods=12, freq='M').strftime('%B'), 1)}

    new_data = pd.DataFrame()

    # Create time series data for each combination of hotel and room type across multiple years
    for year in range(2020, 2024):
        for month in month_dict.values():
            start_date = pd.to_datetime(f'{year}-{month}-01').replace(day=1)
            end_date = pd.to_datetime(f'{year}-{month}-01').replace(day=1) + pd.offsets.MonthEnd(0)
            date_range = pd.date_range(start_date, end_date, freq='D')
            df = pd.DataFrame(date_range, columns=['arrival_date'])
            df['dow'] = df['arrival_date'].dt.day_name()
            df['month'] = df['arrival_date'].dt.month_name()

            # Merge the date data with hotel/room type combinations
            result_df = df.assign(key=1).merge(combinations_df.assign(key=1), on='key').drop('key', axis=1)
            new_data = pd.concat([new_data, result_df], ignore_index=True)

    # Finalize the data by merging with the calculated results
    final_data = pd.merge(new_data, results, how='left', on=['dow', 'hotel', 'room_type', 'month'])
    final_data.replace([np.inf, -np.inf], np.nan, inplace=True)
    final_data = final_data.dropna()
    final_data['room_limit'] = final_data['room_limit'].astype(int)
    final_data['arrival_date'] = pd.to_datetime(final_data['arrival_date']).dt.strftime("%Y-%m-%d %H:%M:%S")
    
    # Write the final modeled data back to Snowflake
    df_model = session.createDataFrame(final_data.values.tolist(), schema=final_data.columns.tolist())
    df_model.write.mode("overwrite").save_as_table("TTH_DB.TTH_REV_OPT_Schema.PROCESSED_PRICING_DATA")

    # Expand the booking data to one row per stay date
    expanded_dfc1 = pd.DataFrame()
    
    for _, row in data.iterrows():
        num_stay_dates = row['total_rns']
        try:
            # Create a row for each stay date
            expanded_bookingc1 = pd.DataFrame({
                'hotel': row['hotel'],
                'room_type': row['reserved_room_type'], 
                'market_segment': row['market_segment'],
                'deposit_type':row['deposit_type'],
                'meal': row['meal'],
                'distribution_channel': row['distribution_channel'],
                'customer_type': row['customer_type'],
                'arrival_date': pd.date_range(start=row['arrival_date_transformed'], periods=num_stay_dates),
                'total_rns': 1,
                'adr': row['adr'],
                'room_limit': row['room_limit']
            })
            
            # Append the stay date information to the new dataframe
            expanded_dfc1 = pd.concat([expanded_dfc1, expanded_bookingc1], ignore_index=True)
        except ValueError as e:
            print(f"Error processing booking for {row['hotel']} on {row['arrival_date_transformed']} : {num_stay_dates} {e}")
    
    # Final processing of booking frequency and writing back to Snowflake
    expanded_dfc1 = expanded_dfc1.sort_values('arrival_date')
    expanded_dfc1 = expanded_dfc1.reset_index(drop=True)
    expanded_dfc1['adr']= np.round(expanded_dfc1['adr'], 2)
    expanded_dfc1['dow'] = expanded_dfc1.arrival_date.dt.strftime('%A')
    expanded_dfc1['month'] = expanded_dfc1.arrival_date.dt.strftime('%B')
    
    booking_frequency = expanded_dfc1.groupby(['hotel','room_type','market_segment','deposit_type','meal',
                                            'distribution_channel','customer_type',
                                               'arrival_date'], as_index=False, sort=True).agg({'adr': 'mean','room_limit': 'mean',
                                                                    'total_rns': 'sum'})
    booking_frequency = booking_frequency.sort_values(by=['arrival_date','hotel','room_type'], ascending=[True, True, True])
    
    # Write booking frequency data back to Snowflake
    booking_frequency["arrival_date"] = pd.to_datetime(booking_frequency["arrival_date"]).dt.strftime("%Y-%m-%d %H:%M:%S")
    df_inter=session.createDataFrame(
            booking_frequency.values.tolist(),
            schema=booking_frequency.columns.tolist())
    df_inter.write.mode("overwrite").save_as_table("TTH_DB.TTH_REV_OPT_Schema.booking_frequency")

    return "Data processed and dumped back to Snowflake"


create_pricing_model(my_session, "BOOKINGS_TRANSFORMED")

Index(['hotel', 'is_canceled', 'lead_time', 'arrival_date_year', 'month',
       'arrival_date_week_number', 'arrival_date_day_of_month',
       'expected_arrival_date', 'reservation_status',
       'reservation_status_date', 'total_stay_nights', 'tally_days',
       'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'children',
       'babies', 'total_guests', 'avg_rooms_per_night', 'total_room_nights',
       'meal', 'country', 'market_segment', 'distribution_channel',
       'previous_cancellations', 'previous_bookings_not_canceled',
       'reserved_room_type', 'assigned_room_type', 'deposit_type',
       'days_in_waiting_list', 'customer_type', 'adr',
       'arrival_date_transformed', 'reservation_status_date_transformed'],
      dtype='object')


KeyError: 'dow'